# 1. Global Dependencies

In [1]:
import math
import time
from typing import List, Dict

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import numpy as np
import scipy.sparse as sp

import warnings

warnings.filterwarnings(
    "ignore",
    category=DeprecationWarning,
)

# 2. Mount Datasets
The datasets are mounted using Google Drive.

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 3. Random Seed

In [2]:
def set_seed(seed):
    np.random.seed(seed)
    torch.cuda.manual_seed(seed)

# 4. Models

GCN - Graph Convolutional Network uses two graph convolution layers and a dropout layer.

SGC - Simple Graph Convolution Network using a simple PyTorch implementation of logistic regression.

Assumption of model SGC model use:
- Features have been preprocessed with k-step graph propagation

In [3]:
class SGC(nn.Module):
    def __init__(self, in_features, out_features):
        super(SGC, self).__init__()
        # setup as fully connected network
        self.fc1 = nn.Linear(in_features, out_features)

    def forward(self, x):
        # forward feed the data through the fc layer
        x = self.fc1(x)
        return x


class GCL(nn.modules.Module):
    def __init__(self, in_features, out_features):
        super(GCL, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = nn.parameter.Parameter(
            torch.FloatTensor(in_features, out_features)
        )
        self.bias = nn.parameter.Parameter(torch.FloatTensor(out_features))
        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1.0 / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        self.bias.data.uniform_(-stdv, stdv)

    def forward(self, x, adj):
        # forward feed the data through the fc layer
        support = torch.mm(x, self.weight)
        output = torch.spmm(adj, support)
        return output + self.bias


class GCN(nn.Module):
    def __init__(self, in_features, in_hidden, out_features, dropout):
        super(GCN, self).__init__()
        self.gc1 = GCL(in_features, in_hidden)
        self.gc2 = GCL(in_hidden, out_features)
        self.dropout = dropout

    def forward(self, x, adj):
        x = self.gc1(x, adj)
        x = F.relu(x)
        x = F.dropout(x, self.dropout, training=self.training)
        x = self.gc2(x, adj)
        return F.log_softmax(x, dim=1)

# 5. Normalization

Graphs should be normalized.

Reference: https://github.com/Tiiiger/SGC/blob/master/normalization.py

In [4]:
def sparse_mx_to_torch_sparse_tensor(sparse_mx):
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(
        np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64)
    )
    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    return torch.sparse_coo_tensor(indices, values, shape)


def aug_normalized_adjacency(adj):
    adj = sp.coo_matrix(adj)
    adj = adj + sp.eye(adj.shape[0])
    row_sum = np.array(adj.sum(1))
    row_sum = (row_sum == 0) * 1 + row_sum
    d_inv_sqrt = np.power(row_sum, -0.5).flatten()
    d_inv_sqrt[np.isinf(d_inv_sqrt)] = 0.0
    d_mat_inv_sqrt = sp.diags(d_inv_sqrt)
    return d_mat_inv_sqrt.dot(adj).dot(d_mat_inv_sqrt).tocoo()


def row_normalize(mx):
    rowsum = np.array(mx.sum(1))
    rowsum = (rowsum == 0) * 1 + rowsum
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.0
    r_mat_inv = sp.diags(r_inv)
    mx = r_mat_inv.dot(mx)
    return mx

# 6. Analysis

Reference: https://github.com/Tiiiger/SGC/blob/master/metrics.py

In [5]:
def accuracy(output, labels):
    preds = output.max(1)[1].type_as(labels)
    correct = preds.eq(labels).double()
    correct = correct.sum()
    return correct / len(labels)

# 7. Loading Datasets

Modified from the reference: https://github.com/Tiiiger/SGC/blob/master/utils.py

Added randomized splits.

In [6]:
def parse_index_file(filename):
    index = []
    for line in open(filename):
        index.append(int(line.strip()))
    return index


def sgc_precompute(features, adj, degree):
    for i in range(degree):
        features = torch.mm(adj, features)
    return features

In [7]:
def load_citation_sgc(path, dataset_str="cora"):
    """Load Citation Networks Datasets"""
    import pickle as pkl
    import networkx as nx

    names = ["x", "y", "tx", "ty", "allx", "ally", "graph"]
    objects = []
    for i in range(len(names)):
        with open(
            "{}/ind.{}.{}".format(path, dataset_str.lower(), names[i]), "rb"
        ) as f:
            objects.append(pkl.load(f, encoding="latin1"))

    x, y, tx, ty, all_x, all_y, graph = tuple(objects)
    test_idx_reorder = parse_index_file(
        "{}/ind.{}.test.index".format(path, dataset_str)
    )
    test_idx_range = np.sort(test_idx_reorder)

    features = sp.vstack((all_x, tx)).tolil()
    features[test_idx_reorder, :] = features[test_idx_range, :]
    adj = nx.adjacency_matrix(nx.from_dict_of_lists(graph))
    adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)

    labels = np.vstack((all_y, ty))
    labels[test_idx_reorder, :] = labels[test_idx_range, :]

    idx_test = test_idx_range.tolist()  # range(500,1500)
    idx_train = range(len(y))  # range(140)
    idx_val = range(len(y), len(y) + 500)  # range(200,500)

    features = row_normalize(features)

    # porting to pytorch
    features = torch.FloatTensor(np.array(features.todense())).float()
    labels = torch.LongTensor(np.where(labels)[1])
    adj = aug_normalized_adjacency(adj)
    adj = sparse_mx_to_torch_sparse_tensor(adj)
    idx_train = torch.LongTensor(idx_train)
    idx_val = torch.LongTensor(idx_val)
    idx_test = torch.LongTensor(idx_test)

    return adj, features, labels, idx_train, idx_val, idx_test

# 8. Train & Test Functions

In [8]:
def train_sgc(model, loss_fn, optimizer, train_features, train_labels):
    # set the mode for the model
    model.train()
    # init optimizer as gradients accumulate
    optimizer.zero_grad()
    # feed forward the data into our network
    output = model(train_features)
    # compute the loss between the output of
    # network and actual label
    loss = loss_fn(output, train_labels)
    # backwards propagation
    loss.backward()
    # move a step on the gradient by optimizer
    optimizer.step()
    # record the loss
    return loss.item()


def test_sgc(model, loss_fn, test_features, test_labels):
    # set the model mode to test
    model.eval()
    with torch.no_grad():
        output = model(test_features)
        acc_val = accuracy(output, test_labels)
    return acc_val


def train_gcn(model, optimizer, features, labels, adj, idx_train, idx_val):
    # set the mode for the model
    model.train()
    # init optimizer as gradients accumulate
    optimizer.zero_grad()
    # feed forward the data into our network
    output = model(features, adj)
    # compute the loss between the output of
    # network and actual label
    loss_train = F.nll_loss(output[idx_train], labels[idx_train])
    # backwards propagation
    loss_train.backward()
    # move a step on the gradient by optimizer
    optimizer.step()
    # record the loss
    loss_val = F.nll_loss(output[idx_val], labels[idx_val])
    return loss_val.item()


def test_gcn(model, features, labels, adj, idx_test):
    # set the model mode to test
    model.eval()
    with torch.no_grad():
        output = model(features, adj)
        acc_val = accuracy(output[idx_test], labels[idx_test])
    return acc_val

# 9. Results Handling

The following function prints out the train and test results in an easy to read table.

In [9]:
from tabulate import tabulate


def print_results_as_table(results_dict):
    table = []
    header = ["Dataset", "Split", "Time (s)", "Test Accuracy"]
    ds_averages = {}
    for ds, splits in results_dict.items():
        ds_total_time = 0.0
        ds_total_acc = 0.0
        num_splits = len(splits)
        for split, data in splits.items():
            table.append([ds, split, data["mean_time"], data["test_acc"]])
            ds_total_time += data["mean_time"]
            ds_total_acc += data["test_acc"]
        ds_averages[ds] = {
            "mean_time": ds_total_time / num_splits,
            "test_acc": ds_total_acc / num_splits,
        }
    for ds, avg_data in ds_averages.items():
        table.append([ds, "Average", avg_data["mean_time"], avg_data["test_acc"]])
    table_format = "grid"
    print(tabulate(table, headers=header, tablefmt=table_format))

# 10. Citation Network Testing for SGC and GCN
This requires the `sgcn_data` folder with the `Cora` and `Pubmed` datasets. `sgcn_data` is just the `data` folder from SGC, reference: https://github.com/Tiiiger/SGC/tree/master/data.

In [21]:
# configuration
seed = 42  # random seed
max_epoch = 100
lr = 0.2  # initial learning rate
weight_decay = 5e-6  # L2 loss on parameters
degree = 2  # degrees of approximation

# set random and torch seed
set_seed(seed)

# setup results dictionary
results_dict = {}
split = "N/A"
# number of loops to average
num_loops = 20

# load the citation data
datapath = "/content/drive/MyDrive/colab_notebooks/ece570/project/sgcn_data/"
dataset_list = ["cora", "pubmed"]

for dataset in dataset_list:
    results_dict[dataset] = {}
    results_dict[dataset][split] = {}
    print(f"** SGC training/testing on dataset {dataset} **")
    time_list = []
    test_acc_list = []
    for loop in range(num_loops):
        loss = 0
        train_acc = 0
        epoch = 1
        adj, features, labels, idx_train, idx_val, idx_test = load_citation_sgc(
            datapath, dataset
        )
        # move to gpu
        features = features.to("cuda")
        adj = adj.to("cuda")
        labels = labels.to("cuda")
        idx_train = idx_train.to("cuda")
        idx_val = idx_val.to("cuda")
        idx_test = idx_test.to("cuda")
        # setup model
        model = SGC(in_features=features.size(1), out_features=labels.max().item() + 1)
        model = model.to("cuda")
        # setup loss and optimizer
        loss_fn = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
        # start training time
        start = time.time()
        # SGC precomputation of features
        features = sgc_precompute(features, adj, degree)
        # train and validate for the number of epochs
        for epoch in range(1, max_epoch + 1):
            loss = loss + train_sgc(
                model, loss_fn, optimizer, features[idx_train], labels[idx_train]
            )
            train_acc = test_sgc(
                model, loss_fn, features[idx_val], labels[idx_val]
            )
        # end training time
        end = time.time()
        # measure test accuracy
        test_acc = test_sgc(model, loss_fn, features[idx_test], labels[idx_test])
        avg_loss = loss / 100
        train_time = end - start
        time_list.append(train_time)
        test_acc_list.append(test_acc.to("cpu"))
    results_dict[dataset][split]["mean_time"] = np.mean(time_list)
    results_dict[dataset][split]["test_acc"] = 100 * np.mean(test_acc_list)

print_results_as_table(results_dict)

** SGC training/testing on dataset cora **
** SGC training/testing on dataset pubmed **
+-----------+---------+------------+-----------------+
| Dataset   | Split   |   Time (s) |   Test Accuracy |
+===========+=========+============+=================+
| cora      | N/A     |   0.186714 |           80.55 |
+-----------+---------+------------+-----------------+
| pubmed    | N/A     |   0.171003 |           77.81 |
+-----------+---------+------------+-----------------+
| cora      | Average |   0.186714 |           80.55 |
+-----------+---------+------------+-----------------+
| pubmed    | Average |   0.171003 |           77.81 |
+-----------+---------+------------+-----------------+


In [22]:
# configuration from pygcn
seed = 42  # random seed
max_epoch = 100
lr = 0.01  # initial learning rate
weight_decay = 5e-4  # L2 loss on parameters
hidden = 50  # number of hidden units
dropout = 0.5  # dropout rate (1 - keep probability)

# set random and torch seed
set_seed(seed)

# setup results dictionary
results_dict = {}
split = "N/A"
# number of loops to average
num_loops = 20

# load the citation data
datapath = "/content/drive/MyDrive/colab_notebooks/ece570/project/sgcn_data/"
dataset_list = ["cora", "pubmed"]

for dataset in dataset_list:
    results_dict[dataset] = {}
    results_dict[dataset][split] = {}
    print(f"** GCN training/testing on dataset {dataset} **")
    time_list = []
    test_acc_list = []
    for loop in range(num_loops):
        loss = 0
        train_acc = 0
        epoch = 1
        adj, features, labels, idx_train, idx_val, idx_test = load_citation_sgc(
            datapath, dataset
        )
        # move to gpu
        features = features.to("cuda")
        adj = adj.to("cuda")
        labels = labels.to("cuda")
        idx_train = idx_train.to("cuda")
        idx_val = idx_val.to("cuda")
        idx_test = idx_test.to("cuda")
        # setup model
        model = GCN(
            in_features=features.shape[1],
            in_hidden=hidden,
            out_features=labels.max().item() + 1,
            dropout=dropout,
        )
        model = model.to("cuda")
        # setup optimizer
        optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
        # start training time
        start = time.time()
        # train and validate GCN
        for epoch in range(1, max_epoch + 1):
            loss = loss + train_gcn(
                model, optimizer, features, labels, adj, idx_train, idx_val
            )
            train_acc = test_gcn(model, features, labels, adj, idx_test)
        # end training time
        end = time.time()
        # measure test accuracy
        test_acc = test_gcn(model, features, labels, adj, idx_test)
        avg_loss = loss / 100
        train_test_time = end - start
        time_list.append(train_test_time)
        test_acc_list.append(test_acc.to("cpu"))
    results_dict[dataset][split]["mean_time"] = np.mean(time_list)
    results_dict[dataset][split]["test_acc"] = 100 * np.mean(test_acc_list)

print_results_as_table(results_dict)

** GCN training/testing on dataset cora **
** GCN training/testing on dataset pubmed **
+-----------+---------+------------+-----------------+
| Dataset   | Split   |   Time (s) |   Test Accuracy |
+===========+=========+============+=================+
| cora      | N/A     |   0.502116 |           81.16 |
+-----------+---------+------------+-----------------+
| pubmed    | N/A     |   0.729225 |           79.27 |
+-----------+---------+------------+-----------------+
| cora      | Average |   0.502116 |           81.16 |
+-----------+---------+------------+-----------------+
| pubmed    | Average |   0.729225 |           79.27 |
+-----------+---------+------------+-----------------+


# 11. Extended Dataset Loading

Running SGC to compare against extended datasets from Geom-GCN and GCNII.

Modified from reference: https://github.com/bingzhewei/geom-gcn/blob/master/utils_data.py

In [14]:
def load_extended_data(path, dataset_name, splits_file_name):
    graph_adjacency_list_file_path = path + dataset_name + "/out1_graph_edges.txt"
    graph_node_features_and_labels_file_path = (
        path + dataset_name + "/out1_node_feature_label.txt"
    )

    import networkx as nx

    G = nx.DiGraph()
    graph_node_features_dict = {}
    graph_labels_dict = {}

    with open(
        graph_node_features_and_labels_file_path, "r", encoding="utf-8"
    ) as graph_node_features_and_labels_file:
        graph_node_features_and_labels_file.readline()
        for line in graph_node_features_and_labels_file:
            line = line.rstrip().split("\t")
            assert len(line) == 3
            assert (
                int(line[0]) not in graph_node_features_dict
                and int(line[0]) not in graph_labels_dict
            )
            graph_node_features_dict[int(line[0])] = np.array(
                line[1].split(","), dtype=np.uint8
            )
            graph_labels_dict[int(line[0])] = int(line[2])

    with open(
        graph_adjacency_list_file_path, "r", encoding="utf-8"
    ) as graph_adjacency_list_file:
        graph_adjacency_list_file.readline()
        for line in graph_adjacency_list_file:
            line = line.rstrip().split("\t")
            assert len(line) == 2
            if int(line[0]) not in G:
                G.add_node(
                    int(line[0]),
                    features=graph_node_features_dict[int(line[0])],
                    label=graph_labels_dict[int(line[0])],
                )
            if int(line[1]) not in G:
                G.add_node(
                    int(line[1]),
                    features=graph_node_features_dict[int(line[1])],
                    label=graph_labels_dict[int(line[1])],
                )
            G.add_edge(int(line[0]), int(line[1]))

    adj = nx.adjacency_matrix(G, sorted(G.nodes()))
    features = np.array(
        [
            features
            for _, features in sorted(G.nodes(data="features"), key=lambda x: x[0])
        ]
    )
    labels = np.array(
        [label for _, label in sorted(G.nodes(data="label"), key=lambda x: x[0])]
    )
    features = row_normalize(features)

    g = adj

    splits_file_path = path + "splits/" + dataset_name + splits_file_name
    with np.load(splits_file_path) as splits_file:
        train_mask = splits_file["train_mask"]
        val_mask = splits_file["val_mask"]
        test_mask = splits_file["test_mask"]

    num_features = features.shape[1]
    num_labels = len(np.unique(labels))
    assert np.array_equal(np.unique(labels), np.arange(len(np.unique(labels))))

    features = torch.FloatTensor(features)
    labels = torch.LongTensor(labels)
    train_mask = torch.BoolTensor(train_mask)
    val_mask = torch.BoolTensor(val_mask)
    test_mask = torch.BoolTensor(test_mask)

    g = aug_normalized_adjacency(g)
    g = sparse_mx_to_torch_sparse_tensor(g)

    return g, features, labels, train_mask, val_mask, test_mask

# 12. Extended Dataset Testing on SGC

This requires the `new_data` folder from Geom-GCN project, reference: https://github.com/bingzhewei/geom-gcn/tree/master/new_data and the `splits` folder, reference: https://github.com/bingzhewei/geom-gcn/tree/master/splits.

Chamelon dataset is added to the `new_data` folder from GCNII project, reference: https://github.com/chennnM/GCNII/tree/master/new_data/chameleon.

In [23]:
# configuration
seed = 42  # random seed
max_epoch = 100
lr = 0.2  # initial learning rate
weight_decay = 5e-6  # L2 loss on parameters
degree = 2  # degrees of approximation

# set random and torch seed
set_seed(seed)

# setup results dictionary
results_dict = {}
# number of loops to average
num_loops = 20

# load the new datasets
datapath = "/content/drive/MyDrive/colab_notebooks/ece570/project/new_data/"
dataset_list = ["chameleon", "cornell", "texas", "wisconsin"]

# load the randomized splits
splits = []
for i in range(10):
    splits.append(f"_split_0.6_0.2_{i}.npz")

for dataset in dataset_list:
    results_dict[dataset] = {}
    print(f"** SGC training/testing on dataset {dataset}")
    for splits_file_name in splits:
        split_offset = splits.index(splits_file_name)
        results_dict[dataset][split_offset] = {}
        # train and test SGC
        time_list = []
        test_acc_list = []
        for loop in range(num_loops):
            loss = 0
            train_acc = 0
            epoch = 1
            adj, features, labels, train_mask, val_mask, test_mask = load_extended_data(
                datapath, dataset, splits_file_name
            )
            # move to gpu
            features = features.to("cuda")
            adj = adj.to("cuda")
            labels = labels.to("cuda")
            train_mask = train_mask.to("cuda")
            val_mask = val_mask.to("cuda")
            test_mask = test_mask.to("cuda")
            # setup model
            model = SGC(
                in_features=features.size(1), out_features=labels.max().item() + 1
            )
            model = model.to("cuda")

            loss_fn = nn.CrossEntropyLoss()
            optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

            start = time.time()
            # SGC precomputation of features
            features = sgc_precompute(features, adj, degree)
            for epoch in range(1, max_epoch + 1):
                loss = loss + train_sgc(
                    model,
                    loss_fn,
                    optimizer,
                    features[train_mask],
                    labels[train_mask]
                )
                train_acc = test_sgc(
                    model, loss_fn, features[val_mask], labels[val_mask]
                )
            test_acc = test_sgc(
                model, loss_fn, features[test_mask], labels[test_mask]
            )
            end = time.time()
            avg_loss = loss / 100
            train_test_time = end - start
            time_list.append(train_test_time)
            test_acc_list.append(test_acc.to("cpu"))
        results_dict[dataset][split_offset]["mean_time"] = np.mean(time_list)
        results_dict[dataset][split_offset]["test_acc"] = 100 * np.mean(test_acc_list)

print_results_as_table(results_dict)

** SGC training/testing on dataset chameleon
** SGC training/testing on dataset cornell
** SGC training/testing on dataset texas
** SGC training/testing on dataset wisconsin
+-----------+---------+------------+-----------------+
| Dataset   | Split   |   Time (s) |   Test Accuracy |
+===========+=========+============+=================+
| chameleon | 0       |   0.240877 |         61.7544 |
+-----------+---------+------------+-----------------+
| chameleon | 1       |   0.226982 |         63.1579 |
+-----------+---------+------------+-----------------+
| chameleon | 2       |   0.221352 |         60.7456 |
+-----------+---------+------------+-----------------+
| chameleon | 3       |   0.202546 |         60.7895 |
+-----------+---------+------------+-----------------+
| chameleon | 4       |   0.198488 |         60.9649 |
+-----------+---------+------------+-----------------+
| chameleon | 5       |   0.200676 |         64.0351 |
+-----------+---------+------------+-----------------+
|

# 13. Extended Dataset Testing on GCN

This requires the `new_data` folder from Geom-GCN project, reference: https://github.com/bingzhewei/geom-gcn/tree/master/new_data and the `splits` folder, reference: https://github.com/bingzhewei/geom-gcn/tree/master/splits.

In [24]:
# configuration from pygcn
seed = 42  # random seed
max_epoch = 100
lr = 0.01  # initial learning rate
weight_decay = 5e-4  # L2 loss on parameters
hidden = 50  # number of hidden units
dropout = 0.5  # dropout rate (1 - keep probability)

# set random and torch seed
set_seed(seed)

# setup results dictionary
results_dict = {}
# number of loops to average
num_loops = 20

# load the citation data
datapath = "/content/drive/MyDrive/colab_notebooks/ece570/project/new_data/"
dataset_list = ["chameleon", "cornell", "texas", "wisconsin"]

# load the randomized splits
splits = []
for i in range(10):
    splits.append(f"_split_0.6_0.2_{i}.npz")

for dataset in dataset_list:
    results_dict[dataset] = {}
    print(f"** GCN training/testing on dataset {dataset} **")
    for splits_file_name in splits:
        split_offset = splits.index(splits_file_name)
        results_dict[dataset][split_offset] = {}
        # train and test SGC
        time_list = []
        test_acc_list = []
        for loop in range(num_loops):
            loss = 0
            train_acc = 0
            epoch = 1
            adj, features, labels, idx_train, idx_val, idx_test = load_extended_data(
                datapath, dataset, splits_file_name
            )
            # move to gpu
            features = features.to("cuda")
            adj = adj.to("cuda")
            labels = labels.to("cuda")
            idx_train = idx_train.to("cuda")
            idx_val = idx_val.to("cuda")
            idx_test = idx_test.to("cuda")
            # setup model
            model = GCN(
                in_features=features.shape[1],
                in_hidden=hidden,
                out_features=labels.max().item() + 1,
                dropout=dropout,
            )
            model = model.to("cuda")
            # setup optimizer
            optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

            # train and test GCN
            start = time.time()
            for epoch in range(1, max_epoch + 1):
                loss = loss + train_gcn(
                    model, optimizer, features, labels, adj, idx_train, idx_val
                )
                train_acc = test_gcn(model, features, labels, adj, idx_test)
            test_acc = test_gcn(model, features, labels, adj, idx_test)
            end = time.time()
            avg_loss = loss / 100
            train_test_time = end - start
            time_list.append(train_test_time)
            test_acc_list.append(test_acc.to("cpu"))
        results_dict[dataset][split_offset]["mean_time"] = np.mean(time_list)
        results_dict[dataset][split_offset]["test_acc"] = 100 * np.mean(test_acc_list)

print_results_as_table(results_dict)

** GCN training/testing on dataset chameleon **
** GCN training/testing on dataset cornell **
** GCN training/testing on dataset texas **
** GCN training/testing on dataset wisconsin **
+-----------+---------+------------+-----------------+
| Dataset   | Split   |   Time (s) |   Test Accuracy |
+===========+=========+============+=================+
| chameleon | 0       |   0.511923 |         56.9627 |
+-----------+---------+------------+-----------------+
| chameleon | 1       |   0.51228  |         58.6513 |
+-----------+---------+------------+-----------------+
| chameleon | 2       |   0.520157 |         53.4211 |
+-----------+---------+------------+-----------------+
| chameleon | 3       |   0.525081 |         59.386  |
+-----------+---------+------------+-----------------+
| chameleon | 4       |   0.509654 |         55.5811 |
+-----------+---------+------------+-----------------+
| chameleon | 5       |   0.522989 |         56.4035 |
+-----------+---------+------------+--------

# 14. Licenses for referenced code

[SGC LICENSE](https://github.com/Tiiiger/SGC/blob/master/LICENSE):
```
The MIT License

Copyright (c) 2019 Tianyi Zhang

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in
all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN
THE SOFTWARE.
```

[Geom-GCN LICENSE](https://github.com/bingzhewei/geom-gcn/blob/master/utils_data.py)
```
#  MIT License
#
#  Copyright (c) 2019 Geom-GCN Authors
#
#  Permission is hereby granted, free of charge, to any person obtaining a copy
#  of this software and associated documentation files (the "Software"), to deal
#  in the Software without restriction, including without limitation the rights
#  to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
#  copies of the Software, and to permit persons to whom the Software is
#  furnished to do so, subject to the following conditions:
#
#  The above copyright notice and this permission notice shall be included in all
#  copies or substantial portions of the Software.
#
#  THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
#  IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
#  FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
#  AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
#  LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
#  OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
#  SOFTWARE.
```